# Predict breast cancer


Predict the class of breast cancer (malignant or ‘bad’ versus benign or ‘good’) from the features of images taken from breast samples. Ten biological attributes of the cancer cell nuclei have been calculated from the images, as described below:
Attribute 	Domain
1. Sample code number 	id number
2. Clump Thickness 	1 - 10
3. Uniformity of Cell Size 	1 - 10
4. Uniformity of Cell Shape 	1 - 10
5. Marginal Adhesion 	1 - 10
6. Single Epithelial Cell Size 	1 - 10
7. Bare Nuclei 	1 - 10
8. Bland Chromatin 	1 - 10
9. Normal Nucleoli 	1 - 10
10. Mitoses 	1 - 10
11. Class 	(2 for benign, 4 for malignant

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [4]:
cancer_prediction = pd.read_csv("cancer.data")

In [9]:
cancer_prediction.head()

,1000025,5,1,1.1,1.2,2,1.3,3,1.4,1.5,2.1
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2
3,1017023,4,1,1,3,2,1,3,1,1,2
4,1017122,8,10,10,8,7,10,9,7,1,4
